In [1]:
import pandas as pd
import numpy as np
import csv
# from surprise import SVD, SVDpp
# from surprise import Dataset
# import pandas as pd
# import surprise
# from surprise import Reader
# from surprise.model_selection import cross_validate
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm
import copy
from sklearn.metrics import roc_auc_score

In [2]:
df_train = pd.read_csv('Data/my_data/train_features.csv')
df_train['country'] = pd.factorize(df_train['s_country'])[0] + 1
df_train = df_train[[i for i in list(df_train.columns) if 's_country' != i]]
df_train = df_train.reindex(columns=[x for x in df_train.columns if x != 'target'] + ['target'])
df_train_copy = copy.deepcopy(df_train)
X = df_train_copy.reindex(columns=[x for x in df_train_copy.columns if x != 'target' and x != 'user' and x != 'song' and x != 'svd_ua'])
    
Y = df_train_copy['target']

C:\Users\yamsharon102\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_test = pd.read_csv('Data/my_data/test_features.csv')

In [4]:
model1 = lgbm.LGBMClassifier(learning_rate=0.05, max_depth=15, 
                             num_leaves=2**8, colsample_bytree=0.7,
                             subsample=0.7)
model1.fit(X, Y, verbose=1)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        importance_type='split', learning_rate=0.05, max_depth=15,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=256, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.7, subsample_for_bin=200000, subsample_freq=0)

In [5]:
model2 = lgbm.LGBMClassifier(learning_rate=0.1, max_depth=15, 
                             num_leaves=2**8, colsample_bytree=0.9,
                             subsample=0.9)
model2.fit(X, Y, verbose=1)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9,
        importance_type='split', learning_rate=0.1, max_depth=15,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=256, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.9, subsample_for_bin=200000, subsample_freq=0)

In [9]:
df_test = pd.read_csv('Data/my_data/test_features.csv', index=False) 

df_test['country'] = pd.factorize(df_test['s_country'])[0] + 1
df_test = df_test[[i for i in list(df_test.columns) if 's_country' != i]]
df_test = df_test.reindex(columns=[x for x in df_test.columns if x != 'target'] + ['target'])

df_test_copy = copy.deepcopy(df_test)
X_test = df_test_copy.reindex(columns=[x for x in df_test_copy.columns if 'Unnamed' not in x and x != 'target' and x != 'user' and x != 'song' and x != 'svd_ua'])    
Y_test = df_test_copy['target']

In [10]:
y1 = model1.predict(X)
print('model1 train AUC',roc_auc_score(Y, y1))
y2 = model2.predict(X)
print('model2 train AUC',roc_auc_score(Y, y2))
y_pred1 = model1.predict(X_test)
print('model1 test AUC',roc_auc_score(Y_test, y_pred1))
y_pred2 = model2.predict(X_test)
print('model2 test AUC',roc_auc_score(Y_test, y_pred2))

model1 train AUC 0.8125542840043058
model2 train AUC 0.8185979767899698
model1 test AUC 0.6709363337608998
model2 test AUC 0.6675493241399761


In [12]:
ensemble_test = [(y_pred1[i]+y_pred2[i])/2 for i in range(len(y_pred1))]
print('ensemble_test test AUC',roc_auc_score(Y_test, ensemble_test))


ensemble_test test AUC 0.6736377247420028
